In [188]:
import numpy as np
from scipy.linalg import expm, sinm, cosm
import qiskit.quantum_info as qi

In [189]:
A = np.array([
    [1.5, 0.5],
    [0.5, 1.5],
    ]).astype('complex')

b = np.array([0, 1]).T

In [219]:
#states
state_zero = np.array([[1.0],[0.0]])
state_one = np.array([[0.0],[1.0]])
#projectors
P0 = np.dot(state_zero, state_zero.T)
P1 = np.dot(state_one, state_one.T)
print(P0)
print(P1)

[[1. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 1.]]


In [302]:
def multi_kron(*args):
    ret = np.array([[1.0]]).astype('complex')
    for q in args:
        ret = np.kron(ret, q)
    return ret

def multi_dot(*args):
    ret = np.eye(np.shape(args[0])[0]).astype('complex')
    for q in args:
        ret = np.dot(ret, q)
    return ret

In [321]:
def U(A, phi):
    return expm(-1j*A*phi).astype('complex')

def H():
    return 1/np.sqrt(2) * np.array([[1, 1],
                                    [1, -1]]).astype('complex')
def X():
    return np.array([[0, 1],
                     [1, 0]]).astype('complex')
def Rz(phi):
    return np.array([[np.exp(-1j*phi/2), 0],
                     [0, np.exp(1j*phi/2)]]).astype('complex')
def Ry(theta):
    return np.array([[np.cos(theta/2), -np.sin(theta/2)],
                     [np.sin(theta/2), np.cos(theta/2)]]).astype('complex')
def I():
    return np.array([[1, 0],
                     [0, 1]]).astype('complex')

#for some reson CONTROL and TARGET are changed
def control(control_q, target_q, n_of_qubits, operator):
    list_of_ops_left = []
    list_of_ops_right = []
    for i in range(n_of_qubits):
        if i == control_q:
            list_of_ops_left.append(P0)
            list_of_ops_right.append(P1)
        elif i == target_q:
            list_of_ops_left.append(I())
            list_of_ops_right.append(operator)
        else:
            list_of_ops_left.append(I())
            list_of_ops_right.append(I())
    return multi_kron(*list_of_ops_left) + multi_kron(*list_of_ops_right).astype('complex')

In [322]:
# orders of qubits changed at all
# one = multi_kron(I(), Ry(np.pi/3))
# two = np.round(control(control_q=1,
#                  target_q=10,
#                  n_of_qubits=2,
#                  operator=U(A, np.pi/4)), 3)
# print(np.round(np.dot(two, one), 3))

# Now real example

In [352]:
#state preparation
state = multi_kron(I(), H(), H(), H(), X())
np.round(state, 3)
#eigenvalues to phases
etp_one = control(control_q=3,
                  target_q=4,
                  n_of_qubits=5,
                  operator=U(A, np.pi/2))
etp_two = control(control_q=2,
                  target_q=4,
                  n_of_qubits=5,
                  operator=U(A, np.pi/4))
etp_three = control(control_q=1,
                  target_q=4,
                  n_of_qubits=5,
                  operator=U(A, np.pi/8))
# print(np.round(multi_dot(state, etp_one, etp_two, etp_three), 3))
#Fourier transform
ft_one = multi_kron(I(), H(), I(), I(), I())
ft_two = control(control_q=1,
                 target_q=3,
                 n_of_qubits=5,
                 operator=Rz(-np.pi/4))
ft_three = control(control_q=1,
                   target_q=2,
                   n_of_qubits=5,
                   operator=Rz(-np.pi/2))
ft_four = multi_kron(I(), I(), H(), I(), I())
ft_five = control(control_q=2,
                  target_q=3,
                  n_of_qubits=5,
                  operator=Rz(-np.pi/2))
ft_six = multi_kron(I(), I(), I(), H(), I())
# print(np.round(multi_dot(state,
#                          etp_one, etp_two, etp_three,
#                          ft_one, ft_two, ft_three, ft_four, ft_five, ft_six), 3))
# #Ancilla qubit
aq_one = control(control_q=1,
                 target_q=0,
                 n_of_qubits=5,
                 operator=Ry(np.pi/2)) 
aq_two = control(control_q=2,
                 target_q=0,
                 n_of_qubits=5,
                 operator=Ry(np.pi/4))
aq_three = control(control_q=3,
                   target_q=0,
                   n_of_qubits=5,
                   operator=Ry(np.pi/8)) 
# print(np.round(multi_dot(
#     aq_three, aq_two, aq_one,
#     ft_six, ft_five, ft_four, ft_three, ft_two, ft_one,
#     etp_three, etp_two, etp_one,
#     state), 3))
# #inverse_fourier
if_one = multi_kron(I(), I(), I(), H(), I())
if_two = control(
    control_q=2,
    target_q=3,
    n_of_qubits=5,
    operator=Rz(np.pi/2))
if_three = multi_kron(I(), I(), H(), I(), I())
if_four = control(control_q=1,
                  target_q=3,
                  n_of_qubits=5,
                  operator=Rz(np.pi/4))
if_five = control(control_q=1,
                  target_q=2,
                  n_of_qubits=5,
                  operator=Rz(np.pi/2))
if_six = multi_kron(I(), I(), I(), H(), I())
if_seven = multi_kron(I(), H(), I(), I(), I())
# print(np.round(multi_dot(
#     if_seven, if_six, if_five, if_four, if_three, if_two, if_one,
#     aq_three, aq_two, aq_one,
#     ft_six, ft_five, ft_four, ft_three, ft_two, ft_one,
#     etp_three, etp_two, etp_one,
#     state), 3))
# #phases to eigenvalues
pte_one = control(control_q=1,
                  target_q=4,
                  n_of_qubits=5,
                  operator=U(A, -np.pi/8))
pte_two = control(control_q=2,
                  target_q=4,
                  n_of_qubits=5,
                  operator=U(A, -np.pi/4))
pte_three = control(control_q=3,
                    target_q=4,
                    n_of_qubits=5,
                    operator=U(A, -np.pi/2))
final_u = multi_dot(
    pte_three, pte_two, pte_one,
    if_seven, if_six, if_five, if_four, if_three, if_two, if_one,
    aq_three, aq_two, aq_one,
    ft_six, ft_five, ft_four, ft_three, ft_two, ft_one,
    etp_three, etp_two, etp_one,
    state)
# print(np.round(final_u, 3))

In [427]:
prime_state = multi_kron(state_zero, state_zero, state_zero, state_zero, state_zero)
final_state = np.dot(final_u, prime_state)
from sklearn.preprocessing import normalize
final_state = final_state / np.linalg.norm(final_state)
# norm2 = normalize(final_state[:,np.newaxis], axis=0).ravel()
rho = np.dot(final_state, final_state.T)

# print(np.round(final_state,2))
print(np.round(rho, 5))

row_sums = rho.sum(axis=1)
rho = rho / row_sums[:, np.newaxis]
# new_matrix

[[-0.01363-0.03243j  0.01251+0.02775j  0.02368-0.00929j ...
   0.0155 +0.00241j  0.00954-0.01902j -0.01093-0.03465j]
 [ 0.01251+0.02775j -0.01144-0.02372j -0.02028+0.00856j ...
  -0.01346-0.00174j -0.00783+0.01666j  0.01023+0.02973j]
 [ 0.02368-0.00929j -0.02028+0.00856j  0.0063 +0.01728j ...
  -0.00201+0.01116j  0.01358+0.00723j  0.02524-0.0073j ]
 ...
 [ 0.0155 +0.00241j -0.01346-0.00174j -0.00201+0.01116j ...
  -0.00454+0.00532j  0.00499+0.00807j  0.01571+0.00395j]
 [ 0.00954-0.01902j -0.00783+0.01666j  0.01358+0.00723j ...
   0.00499+0.00807j  0.01249-0.0031j   0.01162-0.01865j]
 [-0.01093-0.03465j  0.01023+0.02973j  0.02524-0.0073j  ...
   0.01571+0.00395j  0.01162-0.01865j -0.00795-0.03668j]]


In [424]:
prob0 = np.trace(np.dot(multi_kron(I(), I(), I(), I(), P1), rho))
prob0

(0.5288657755275488-0.02934457121413475j)

In [344]:
# Simulate
def simulate(n, shots):
    ret = 0
    for shot in range(shots):
        if np.random.rand() < prob0:
            ret += 0
            state_ret = np.dot(multi_kron(P0, gate_I), state)
        else:
            ret = 1
            state_ret = np.dot(multi_kron(P1, gate_I), state) 
    return state_ret/n

In [178]:
#density matrix
rho = DensityMatrix.from_instruction(qc).data
state = Statevector.from_instruction(qc).data

In [ ]:
# Normalize
from scipy import linalg
prob0_0 = np.trace(np.dot(multi_kron(P0, P1), rho))
print(prob0_0)

In [230]:
from qiskit.quantum_info import SparsePauliOp, DensityMatrix, Statevector
from qiskit.opflow.list_ops import SummedOp
from qiskit.circuit import Parameter
from qiskit.opflow import I, X, Y, Z, H, CX, Zero, ListOp, PauliExpectation, PauliTrotterEvolution, CircuitSampler, MatrixEvolution, Suzuki, PauliSumOp
from qiskit.circuit import ControlledGate
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, execute, Aer, transpile
from qiskit.quantum_info.operators import Operator
from qiskit.extensions import UnitaryGate
from qiskit.tools.visualization import plot_histogram

from qiskit.synthesis import MatrixExponential
from qiskit.quantum_info import Operator

In [96]:
n=4
pauli_op = PauliSumOp(SparsePauliOp.from_operator(A))
phi = Parameter('ϕ')
evolution_op = (phi * pauli_op).exp_i() # exp(-iϕA)
trotterized_op = PauliTrotterEvolution(trotter_mode=Suzuki(order=2, reps=1)).convert(evolution_op).bind_parameters({phi: np.pi/n})
trotterized_op.to_matrix()
gate = trotterized_op.to_circuit()
gate.name = f"e^(i*A*pi/{n})"
gate.label = f"e^(i*A*np.pi/{n})"
gate = gate.to_gate().control()

In [187]:
n_b = 1
n_ = 0
n_ancilla = 1
n_cl = 2

qc = QuantumCircuit(n_b + n_ + n_ancilla, n_cl)
# Matrix exponentiation
qc.ry(np.pi/3, 0)
qc.h(1)
# qc.cry(np.pi/4, 0, 1)
qc.cx(0,1)
print(qc)
op = qi.Operator(qc).data
print(op)
# np.round(op, 2)

     ┌─────────┐     
q_0: ┤ Ry(π/3) ├──■──
     └──┬───┬──┘┌─┴─┐
q_1: ───┤ H ├───┤ X ├
        └───┘   └───┘
c: 2/════════════════
                     
[[ 0.61237244+0.j -0.35355339+0.j  0.61237244+0.j -0.35355339+0.j]
 [ 0.35355339+0.j  0.61237244+0.j -0.35355339+0.j -0.61237244+0.j]
 [ 0.61237244+0.j -0.35355339+0.j -0.61237244+0.j  0.35355339+0.j]
 [ 0.35355339+0.j  0.61237244+0.j  0.35355339+0.j  0.61237244+0.j]]


In [179]:
qc.measure([0,1], [0,1])
simulator = Aer.get_backend('aer_simulator')
qc = transpile(qc, simulator)
# Run and get counts
shots=1024
result = simulator.run(qc, shots=shots).result()
counts = result.get_counts(qc)
for k, v in counts.items():
    print(f'{k}: {v/1024}')

11: 0.126953125
00: 0.3623046875
01: 0.13671875
10: 0.3740234375


In [181]:
#probabilities
def multi_kron(*args):
    ret = np.array([[1.0]])
    for q in args:
        ret = np.kron(ret, q)
    return ret

gate_I = np.array([[1, 0],[1, 0]])

(0.12499999999999996+0j)
